In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../../')
print(os.getcwd())
from v2.strategy.indicators.optimal_v2 import Optimal_v2
from v2.model import Trading
from v2.strategy.indicators.param import Param
from v2.strategy.indicators.roc import RateOfChange
from v2.research.scripts.notebook_utils import notebookUtils
from v2.research.scripts.scoreboard import updateScoreboard
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from v2.strategy.strategies.atlas.atlas_v1_2 import Atlas_v1_2
from v2.strategy.strategies.atlas.atlas_v0_0 import Atlas_v0_0

/home/marshingjay/Repos/moonshot/back_testing


In [2]:
nu = notebookUtils()
model_name = "skunk"

In [3]:
def simulate_all_trades(dataset, strategy):
    dataset['is_potential_buy'] = False
    coin = 'COIN'
    generic_strategy = strategy([coin])
    opened_trades = dict()
    closed_trades = dict()
    for row in dataset.itertuples():
        otkeys = list(opened_trades.keys())
        for tick in otkeys:
            opened_trades[tick][0].process(row, coin)
            if opened_trades[tick][0].calc_exit(row, coin):
                closed_trades[tick] = (row.close/opened_trades[tick][1]) - 1
                del opened_trades[tick]
        if generic_strategy.calc_entry(row, coin):
            dataset.loc[(dataset.time == row.time),'is_potential_buy']=True
            opened_trades[row.time] = (generic_strategy, row.close)
            generic_strategy = strategy([coin])
        
    dataset['simul_profit'] = 0.0
    for k in closed_trades:
        dataset.loc[(dataset.time == k),'simul_profit']=closed_trades[k]

In [4]:
datasets, features, indicators = nu.loadData(indicators=[('SMA','close',''),('SMA','close','for_short'),('roc','SMA_for_short', 'shorter')],
                            param_spec={
                                'SMA':{'period':150}, 
                                'SMA_for_short':{'period':30}, 
                                'RateOfChange_shorter':{'period':45},
                                },
                            spans=[],
                            seperate_by_coin=True)

Loading data from ONE...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from ETC...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from chunk 5...
Loading data from chunk 6...
Loading data from chunk 7...
Loading data from ONT...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from chunk 5...
Loading data from chunk 6...
Loading data from chunk 7...
Loading data from BCH...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from RVN...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from EOS...
Loading data from chunk 0...
Loading d

In [7]:
for d in datasets:
    simulate_all_trades(d, Atlas_v0_0)
dataset = pd.concat(datasets)

buys = dataset[dataset['is_potential_buy'] == True]
average_profit = sum(buys['simul_profit']) / len(buys['simul_profit'])
average_profit

KeyboardInterrupt: 

In [49]:
model_dataset, features, indicators = nu.loadData(indicators=[('BOP', 'close', ''), ('PSAR', 'close', '')],
                            param_spec={},
                            
                            spans=[
                                    {'indicator_name':'NATR',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [300]},
                                    {'indicator_name':'CMO',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [10, 30, 200]},
                                    {'indicator_name':'CCI',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [30, 200]},
                                    {'indicator_name':'WILLR',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [30, 200]},
                                    
                                    ],

                            scale='minmaxwindow')

Loading data from ZEC...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from ZRX...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from ATOM...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from DASH...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from ONE...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from BAT...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading

In [50]:
joined_dataset = model_dataset.merge(dataset, on=['time', 'open', 'high', 'low', 'close', 'volume'])
joined_dataset = joined_dataset[joined_dataset['is_potential_buy'] == True]

joined_dataset.drop(["time", "open", "high", "volume", "low", "SMA", "SMA_for_short", "RateOfChange_shorter", "is_potential_buy"], axis=1, inplace=True)

joined_dataset.dropna(inplace=True)


In [51]:
joined_dataset["optimal"] = joined_dataset.apply(lambda x: nu.filter_optimal(x.simul_profit, (average_profit, 1.0), 'buy'),  axis=1)
joined_dataset.drop(["simul_profit"], axis=1, inplace=True)

In [56]:
trainX, testX, trainy, testy = nu.splitData(joined_dataset, split_size=0.2, y_column_name="optimal", shuffle_data=False, balance_unbalanced_data=True, balance_info={'multiplier_val':1, 'superset_class_val':0, 'randomize_concat':True})
len(trainX)

In [58]:
class_weights = nu.getWeights(trainy)
class_weights

In [59]:
clf = RandomForestClassifier(random_state=69420, class_weight=class_weights, n_jobs=-1, n_estimators=100, criterion='entropy', min_samples_leaf=5)
clf.fit(trainX.drop("close", axis=1).values, trainy.values)
print(clf.score(testX.drop("close", axis=1).values, testy.values))

0.6075924724205062


In [60]:
feature_importances = permutation_importance(clf, trainX.drop("close", axis=1).values, trainy.values, n_repeats=10, random_state=69420)

for i in range(len(feature_importances.importances_mean)):
    print('i: {}, Feature: {}, Score: {}'.format(i, trainX.drop("close", axis=1).columns[i], feature_importances.importances_mean[i]))

i: 0, Feature: BOP, Score: 0.013147946850106274
i: 1, Feature: PSAR, Score: 0.08472183915505298
i: 2, Feature: NATR_close_period_300, Score: 0.21238704025179683
i: 3, Feature: CMO_close_period_10, Score: 0.09142073754400765
i: 4, Feature: CMO_close_period_30, Score: 0.10483800314746013
i: 5, Feature: CMO_close_period_200, Score: 0.22236805814689226
i: 6, Feature: CCI_close_period_30, Score: 0.142795723347989
i: 7, Feature: CCI_close_period_200, Score: 0.1627674935509515
i: 8, Feature: WILLR_close_period_30, Score: 0.06921654201210312
i: 9, Feature: WILLR_close_period_200, Score: 0.12353132047309245


In [62]:
# model_version = nu.exportModel(clf, 'playmaker', new_version=False, indicators=indicators, features=features, proba_threshold=0.5, is_nn=False)